<a href="https://colab.research.google.com/github/MariamDundua/Mariam-Dundua/blob/master/t5_base_training_on_QA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training

In [ ]:
!nvidia-smi

In [ ]:
!pip install grpcio-tools==1.34.1
!pip install git+https://github.com/deepset-ai/haystack.git

In [ ]:
#!pip install --quiet transformers==4.1.1
!pip install --quiet pytorch-lightning==1.1.3
#!pip install pytorch-lightning
!pip install --quiet tokenizers==0.9.4
!pip install --quiet sentencepiece==0.1.94

In [ ]:
!pip install datasets transformers

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!apt install git-lfs

In [ ]:
!pip install torchtext==0.8.0 torch==1.7.1 pytorch-lightning==1.1.3


In [ ]:
!pip install transformers
!pip install PyDrive


In [ ]:
import transformers

print(transformers.__version__)

In [ ]:
import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation

import pandas as pd
import numpy as np
import torch
from pathlib import Path
from torch.utils.data import Dataset,DataLoader
import pytorch_lightning as pl
#import pytorch_lightning as pl

from sklearn.model_selection import train_test_split
from termcolor import colored
import textwrap
from transformers import (AdamW,
                          T5ForConditionalGeneration,
                          T5Tokenizer,
                          get_linear_schedule_with_warmup)




In [ ]:
pl.seed_everything(42)

Data

In [ ]:
import json
from google.colab import files
uploaded=files.upload()
dt=(uploaded['answers.json'])
dataa = json.loads(dt)
dataa

In [ ]:
import pandas as pd
data=pd.DataFrame(columns=['context','question','answers','answer_start','document_id','question_id','is_impossible'])

In [ ]:
for i in range(120):
  for j in range(len(dataa['data'][i]['paragraphs'][0]['qas'])):

    data=data.append({'question':dataa['data'][i]['paragraphs'][0]['qas'][j]['question'],
                      'is_impossible':dataa['data'][i]['paragraphs'][0]['qas'][j]['is_impossible'],
                      'answers':dataa['data'][i]['paragraphs'][0]['qas'][j]['answers'][0]['text'],
                      'answer_start':dataa['data'][i]['paragraphs'][0]['qas'][j]['answers'][0]['answer_start'],
                      'document_id':dataa['data'][i]['paragraphs'][0]['qas'][j]['answers'][0]['document_id'],
                      'id':dataa['data'][i]['paragraphs'][0]['qas'][j]['answers'][0]['question_id'],
                      'context':dataa['data'][i]['paragraphs'][0]['context']   
                      },ignore_index = True)
data['id']=data['id'].astype('object')   
data['document_id']=data['document_id'].astype('object')   

In [ ]:
data['answer_len']=[len(i) for i in data['answers'].tolist()]


In [ ]:
data['answer_end']=data['answer_start']+data['answer_len']

In [ ]:
license=[239179,239180,239181,239183,239184,239185]
dt=pd.DataFrame(columns=['context','question','answers','answer_start','document_id','question_id','is_impossible','answer_end','answer_len'])
for i in license:
  dt=dt.append(data[data['document_id']==i])

In [ ]:
data.drop(dt.index,axis=0,inplace=True)

In [ ]:
def color_answer(qest_ans):
  answer_start,answer_end=qest_ans['answer_start'],qest_ans['answer_end']
  context=qest_ans['context']
  return colored(context[:answer_start],'white')+ colored(context[answer_start:answer_end+1],'blue')+colored(context[answer_end+1:],'white')


In [ ]:
from termcolor import colored
import textwrap
print("Question:{}".format(data.iloc[2].question))
print('==================================')
print('==================================')
print('==================================')
print('')
#color_answer(data.iloc[0])
for wrap in textwrap.wrap(color_answer(data.iloc[2]),width=100):
  print(wrap)

Tokenizer

In [ ]:
#model_t5='t5-large'
model_t5='t5-base'

In [ ]:
tokenizer=T5Tokenizer.from_pretrained(model_t5)

In [ ]:
class legal_dataset(Dataset):
  def __init__(self,
               data: pd.DataFrame,
               tokenizer:T5Tokenizer,
               source_max_token_length: int=1000,
               target_max_token_length: int=400):
    self.tokenizer=tokenizer
    self.data=data
    self.source_max_token_length=source_max_token_length
    self.target_max_token_length=target_max_token_length

  def __len__(self):
    return len(self.data)  
  def __getitem__(self,index: int):
    data_row=self.data.iloc[index]  
    source_encoding=tokenizer(
      data_row['question'],
      data_row['context'],
      max_length=self.source_max_token_length,
      padding='max_length',
      truncation='only_second',
      return_attention_mask=True,
      add_special_tokens=True,
      return_tensors='pt')
    target_encoding=tokenizer(
      data_row['answers'],
      max_length=self.target_max_token_length,
      padding='max_length',
      truncation=True,
      return_attention_mask=True,
      add_special_tokens=True,
      return_tensors='pt')
    
    labels=target_encoding['input_ids']
    labels[labels==0]=-100
    return dict(
        question=data_row['question'],
        context=data_row['context'],
        answer_text=data_row['answers'],
        input_ids=source_encoding['input_ids'].flatten(),
        attention_mask=source_encoding['attention_mask'].flatten(),
        labels=labels.flatten()
    )

In [ ]:
class legal_QA_model(pl.LightningDataModule):
  def __init__(self,train_df:pd.DataFrame,
               test_df:pd.DataFrame,
               tokenizer: T5Tokenizer,
               batch_size:int=8,
               source_max_token_length: int=1000,
               target_max_token_length: int=400):
    super().__init__()
    self.batch_size=batch_size
    self.source_max_token_length=source_max_token_length
    self.target_max_token_length=target_max_token_length
    self.tokenizer=tokenizer
    self.train_df=train_df
    self.test_df=test_df
  def setup(self):
    self.train_dataset=legal_dataset(
        self.train_df,
        self.tokenizer,
        self.source_max_token_length,
        self.target_max_token_length)
    self.test_dataset=legal_dataset(
        self.test_df,
        self.tokenizer,
        self.source_max_token_length,
        self.target_max_token_length )
  def train_dataloader(self):
    return DataLoader(self.train_dataset,
                      batch_size=self.batch_size,
                      shuffle=True,
                      num_workers=4)
  def val_dataloader(self):
    return DataLoader(self.test_dataset,
                      batch_size=1,
                      
                      num_workers=4)
  def val_dataloader(self):

    return DataLoader(self.test_dataset,
                      batch_size=1,
                      
                      num_workers=4)        

In [ ]:
BATCH_SIZE=1
N_EPOCHS=20
data_module=legal_QA_model(data,dt,tokenizer,BATCH_SIZE)
data_module.setup()

In [ ]:
data_module

Training

In [ ]:
class Legal_QA(pl.LightningModule):
  def __init__(self):
    super().__init__()
    self.model=T5ForConditionalGeneration.from_pretrained(model_t5,return_dict=True)


  def forward(self,input_ids, attention_mask,labels=None):
    output=self.model(input_ids=input_ids,
                 attention_mask=attention_mask,labels=labels)
    
    return output.loss, output.logits

  def training_step(self,batch,batch_idx):
    input_ids=batch['input_ids']
    attention_mask=batch['attention_mask']
    labels=batch['labels']
    loss,outputs=self.forward(input_ids, attention_mask,labels)
    self.log('train_loss',loss,prog_bar=True,logger=True)
    return loss
  def validation_step(self,batch,batch_idx):
    input_ids=batch['input_ids']
    attention_mask=batch['attention_mask']
    labels=batch['labels']
    loss,outputs=self.forward(input_ids, attention_mask,labels)
    self.log('val_loss',loss,prog_bar=True,logger=True)
    return loss
  def test_step(self,batch,batch_idx):
    input_ids=batch['input_ids']
    attention_mask=batch['attention_mask']
    labels=batch['labels']
    loss,outputs=self.forward(input_ids, attention_mask,labels)
    self.log('test_loss',loss,prog_bar=True,logger=True)
    return loss
  def configure_optimizers(self):
    return AdamW(self.parameters(),lr=0.0001)

In [ ]:
my_model=Legal_QA()

In [ ]:
from pytorch_lightning.callbacks import ModelCheckpoint
checkpoint_callback=ModelCheckpoint(
    dirpath='checkpoints',
    filename='best-checkpoints',
    save_top_k=1,
    verbose=True,
    monitor='val_loss',
    mode='min'
)

In [ ]:
from pytorch_lightning.loggers import TensorBoardLogger

logger=TensorBoardLogger('training-logs',name='legal-QA')
trainer=pl.Trainer(
    logger=logger,
    checkpoint_callback=checkpoint_callback,
    max_epochs=N_EPOCHS,
    gpus=1,
    progress_bar_refresh_rate=30
)

In [ ]:
%load_ext tensorboard


In [ ]:
%tensorboard --logdir ./training-logs

In [ ]:
!rm -rf training_logs
!rm -rf lightning_logs

In [ ]:
trainer.fit(my_model,data_module)

In [ ]:
trainer.fit(my_model,data_module)

In [ ]:
trainer.test()
trained_model=Legal_QA.load_from_checkpoint('checkpoints/best-checkpoints.ckpt')
trained_model.freeze()


In [ ]:
#save model
torch.save(trained_model,'/content/t5modell_n')

#Loading trained model

In [ ]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!7z e /content/drive/MyDrive/model_t5.7z

In [ ]:
import torch
model_up = torch.load('/content/t5modell')

In [ ]:
def generate_aswer(question,context):
  source_encoding=tokenizer(
    question,
    context,
    max_length=1000,
    padding='max_length',
    truncation='only_second',
    return_attention_mask=True,
    add_special_tokens=True,
    return_tensors='pt')

  generated_ids=model_up.model.generate(
      
      input_ids=source_encoding['input_ids'],
      attention_mask=source_encoding['attention_mask'],
      num_beams=1,
      max_length=400,
      repetition_penalty=2.5,
      length_penalty=1.0,
      early_stopping=True,
      use_cache=True)
  pred=[tokenizer.decode(generated_id,skip_special_tokens=True,clean_up_tokenization_spaces=True) for generated_id in generated_ids]
  return " ".join(pred)                     

In [ ]:
ans=[]
for i in range(2):  
  ans.append(generate_aswer(dt.iloc[0]).question,dt.iloc[0]).context)
  print(ans[-1])

In [ ]:
real=[i for i in dt['answers'].tolist()]

In [ ]:
%%shell
jupyter nbconvert --to html /content/t5_base_train.ipynb